# LightGlue Demo
In this notebook we match two pairs of images using LightGlue with early stopping and point pruning.

In [1]:
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import cv2

torch.set_grad_enabled(False)

## Load extractor and matcher module
In this example we use SuperPoint features combined with LightGlue.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'
print(device)
extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)

cuda


In [49]:
def pytorch_infer(image0_path: str, image1_path: str) -> None:
    image0 = load_image(image0_path)[0]
    image1 = load_image(image1_path)[0]

    feats0 = extractor.extract(image0.to(device))
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})

    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension

    # Convert keypoints from PyTorch to OpenCV format
    def convert_to_cv_keypoints(keypoints):
        cv_keypoints = []
        for kp in keypoints:
            cv_keypoints.append(cv2.KeyPoint(kp[0].item(), kp[1].item(), 1))  # (x, y, size)
        return cv_keypoints

    cv_kpts0 = convert_to_cv_keypoints(feats0["keypoints"])
    cv_kpts1 = convert_to_cv_keypoints(feats1["keypoints"])

    # Convert matches from PyTorch to OpenCV format
    def convert_to_cv_matches(matches):
        cv_matches = []
        for match in matches:
            cv_matches.append(cv2.DMatch(match[0].item(), match[1].item(), 0))  # (queryIdx, trainIdx, distance)
        return cv_matches

    cv_matches = convert_to_cv_matches(matches01["matches"])

    match_image = cv2.drawMatches(cv2.imread(image0_path), cv_kpts0, cv2.imread(image1_path), cv_kpts1, cv_matches, None)

    cv2.putText(img=match_image, text=f'Number of matches: {len(cv_matches)}', 
                org=(10, 30), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(255, 255, 255),
                thickness=3,
                lineType=4)
    cv2.imwrite(f"match_result_image.png", match_image)

## Easy example
The top image shows the matches, while the bottom image shows the point pruning across layers. In this case, LightGlue prunes a few points with occlusions, but is able to stop the context aggregation after 4/9 layers.

In [50]:
IMAGE_0_PATH = "assets/DSC_0410.JPG"
IMAGE_1_PATH = "assets/DSC_0411.JPG"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

## Difficult example
For pairs with significant viewpoint- and illumination changes, LightGlue can exclude a lot of points early in the matching process (red points), which significantly reduces the inference time.

In [51]:
IMAGE_0_PATH = "/workspace/image/lm81-1.png"
IMAGE_1_PATH = "/workspace/image/lm81-2.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

In [52]:
!ls -lht  ../../image/

total 9.1M
-rwxrwxrwx 1 root root 1.2M Jul 16 07:28 satelite.png
-rwxrwxrwx 1 root root 1.7M Jul 16 07:28 google_map.png
-rwxrwxrwx 1 root root 2.3M Jul 16 07:27 lm81-2.png
-rwxrwxrwx 1 root root 492K Jul 16 07:27 lm81-1.png
-rwxrwxrwx 1 root root 131K Jul 16 07:24 webcam-Oxford-797-12.jpg
-rwxrwxrwx 1 root root 254K Jul 16 07:24 sacre_coeur2.jpg
-rwxrwxrwx 1 root root 197K Jul 16 07:24 sacre_coeur1.jpg
drwxrwxrwx 1 root root 4.0K Jul 15 17:14 freiburg_sequence
-rwxrwxrwx 1 root root 465K Jul  2 14:28 image0.png
-rwxrwxrwx 1 root root 464K Jul  2 14:28 image1.png
-rwxrwxrwx 1 root root 219K Jul  2 14:28 match_image.png
-rwxrwxrwx 1 root root 1.8M Jul  2 14:28 superpoint_superglue_tensorrt.gif


In [53]:
IMAGE_0_PATH = "/workspace/image/freiburg_sequence/1341847980.722988.png"
IMAGE_1_PATH = "/workspace/image/freiburg_sequence/1341847990.810771.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

In [54]:
IMAGE_0_PATH = "/workspace/image/freiburg_sequence/1341847980.722988.png"
IMAGE_1_PATH = "/workspace/image/freiburg_sequence/1341847987.758741.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

In [56]:
IMAGE_0_PATH = "/workspace/image/google_map.png"
IMAGE_1_PATH = "/workspace/image/satelite.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

In [60]:
IMAGE_0_PATH = "/workspace/image/google_map.png"
IMAGE_1_PATH = "/workspace/image/satelite_crop.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)

In [61]:
IMAGE_0_PATH = "/workspace/image/google_map.png"
IMAGE_1_PATH = "/workspace/image/satelite_crop2.png"
pytorch_infer(IMAGE_0_PATH, IMAGE_1_PATH)